In [37]:
import spacy
import pandas as pd
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [38]:
df = pd.read_csv('data/articles_topics_conflict_cleaned.csv', parse_dates=['date'])

In [39]:
df

,summary,date,chat_GPT_location,lat,lng,article_id,cd_district_name,hunger,refugees,humanitarian,conflict
0,The article discusses the readiness of Guit co...,2011-07-07,Bentiu,9.231487,29.800503,88,Rubkona,False,False,False,True
1,The article discusses the celebration of South...,2011-07-09,Bentiu,9.231487,29.800503,103,Rubkona,False,False,False,True
2,The article discusses the congratulatory messa...,2011-07-11,Bentiu,9.231487,29.800503,221,Rubkona,False,False,False,True
3,The article discusses the killing of rebel lea...,2011-07-23,Koch county,8.684727,29.881520,299,Koch,True,True,True,True
4,The article discusses the killing of South Sud...,2011-07-23,Unity state,8.927721,29.788925,327,Koch,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
454,The article discusses how the UK government ha...,2022-12-09,Koch County,8.684727,29.881520,18396,Koch,True,True,True,True
455,The article discusses how the humanitarian pri...,2022-08-22,Unity State,8.927721,29.788925,18398,Koch,True,True,True,True
456,The article discusses the intensification of a...,2022-09-26,Unity State,8.927721,29.788925,18427,Koch,False,False,False,True
457,The article discusses a UN report documenting ...,2022-09-06,Unity state,8.927721,29.788925,18439,Koch,True,True,True,True


In [43]:
df['summary'][:231]

0      The article discusses the readiness of Guit co...
1      The article discusses the celebration of South...
2      The article discusses the congratulatory messa...
3      The article discusses the killing of rebel lea...
4      The article discusses the killing of South Sud...
                             ...                        
226    The article discusses the seizure of Bentiu, t...
227    The article discusses recent violent attacks o...
228    The article discusses the South Sudanese army ...
229    The article discusses the conflict in South Su...
230    The article discusses the increasing number of...
Name: summary, Length: 231, dtype: object

# use for Article Assignment investigation

In [66]:
index = 240

tex = df['summary'][index]

display(df.iloc[index,:][['chat_GPT_location', 'cd_district_name']])



text1 = NER(tex)

orgs_dict = {}

for word in text1.ents:
    if word.label_ == 'ORG':
        org_name = word.text
        if org_name in orgs_dict:
            orgs_dict[org_name] += 1
        else:
            orgs_dict[org_name] = 1

# Display the organizations and their respective counts
print(f"Organizations mentioned in the article:\n")
for org, count in orgs_dict.items():
    print(f"{org}: {count}")

displacy.render(text1,style="ent",jupyter=True)

chat_GPT_location     Bentiu
cd_district_name     Rubkona
Name: 240, dtype: object

Organizations mentioned in the article:

Unity: 1
Bentiu Mosque: 1


## example

In [26]:
orgs_dict = {}

for word in text1.ents:
    if word.label_ == 'ORG':
        org_name = word.text
        if org_name in orgs_dict:
            orgs_dict[org_name] += 1
        else:
            orgs_dict[org_name] = 1

# Display the organizations and their respective counts
print(f"Organizations mentioned in the article:\n")
for org, count in orgs_dict.items():
    print(f"{org}: {count}")


Organizations mentioned in the article:

the South Sudan Human Rights Society: 1
SSHURSA: 2
the South Sudan National Legislative Assembly: 1
Justice: 1


In [28]:
orgs = set()

for word in text1.ents:
    if word.label_ == 'ORG':
        orgs.add(word.text)

display(f'Orgs mentioned in article: {orgs}')

"Orgs mentioned in article: {'Justice', 'SSHURSA', 'the South Sudan Human Rights Society', 'the South Sudan National Legislative Assembly'}"

In [29]:
# source: https://www.analyticsvidhya.com/blog/2021/06/nlp-application-named-entity-recognition-ner-in-python-with-spacy/
display(spacy.explain("ORG"))
display(spacy.explain("GPE"))
displacy.render(text1,style="ent",jupyter=True)

'Companies, agencies, institutions, etc.'

'Countries, cities, states'